<div style="line-height:0.45">
<h1 style="color:#F10C7F  "> Decision Trees 1 </h1>
</div>
<div style="line-height:0.5">
<h4> Decision Trees implementation in numpy from scratch.
</h4>
</div>
<div style="margin-top: 4px;">
<span style="display: inline-block;">
    <h3 style="color: lightblue; display: inline;">Keywords:</h3> functools update class + Counter + __name__ + globals()
</span>
</div>

In [1]:
from collections import Counter
import numpy as np
import functools

In [2]:
def update_class(main_class=None, exclude=("__module__", "__name__", "__dict__", "__weakref__")):
    """ Class decorator.\\
    Adds all methods and members from the wrapped class to main_class, to allow write in various cell methods of same class.
    
    Args:
        - main_class: class to which to append members. Defaults to the class with the same name as the wrapped class
        - exclude: black-list of members which should not be copied
    """
    def decorates(main_class, exclude, appended_class):
        if main_class is None:
            main_class = globals()[appended_class.__name__]
        for k, v in appended_class.__dict__.items():
            if k not in exclude:
                setattr(main_class, k, v)
        return main_class

    return functools.partial(decorates, main_class, exclude)

In [3]:
class DecisionTree:
    """ Decision Tree Class.

    Args:
        - Maximum depth of the decision tree [int]
        - Minimum number of instances required to create a node [int]

    Attributes:
        - Maximum depth of the decision tree [int]
        - Minimum number of instances required to create a node [int]
        - Final decision tree structure [dict]
        """
    def __init__(self, max_depth, min_node_size):
        """ Constructor to initialize the DecisionTree object with the specified maximum depth and minimum node size. """
        self.max_depth = max_depth
        self.min_node_size = min_node_size
        self.final_tree = {}

    def calculate_gini_index(self, child_nodes):
        """ Calculate the Gini index (the impurity in a node) for a set of child nodes\\
        It is used to evaluate the quality of a split.

    Parameters:
        List of child nodes

    Details:
        - Initialize the total_instances variable to 0
        - Iterate over each child node to calculate the total number of instances in the parent node
        - Initialize the gini_index variable to 0
        - For each child node:
            - Calculate the number of instances in the current child node
            - If the child node is empty, skip the calculation to avoid division by zero
            - Create a list (class_values) to store the class values of instances in the current node
            - Count the frequency of each class value using the Counter class
            - Calculate the Gini index for the current node by subtracting the squared frequency of each class value
            - Update the node_gini variable
            - Add the weighted node Gini index to the gini_index variable based on the number of instances in the node
    
    Returns:
        Gini index [float]
    """
        total_instances = 0
        for node in child_nodes:
            total_instances += len(node)
        gini_index = 0
        for node in child_nodes:
            num_instances = len(node)
            if num_instances == 0:
                continue
            class_values = []
            for row in node:
                class_values.append(row[-1])
            class_freq = Counter(class_values).values()
            node_gini = 1
            for freq in class_freq:
                node_gini -= (freq / num_instances) ** 2
            gini_index += (num_instances / total_instances) * node_gini
        return gini_index

In [4]:
@update_class()
class DecisionTree:
    def apply_split(self, feature_idx, threshold, data):
        """ Split to the data based on a given feature index and threshold.

    Parameters:
        - Index of the feature to split on [int].
        - Threshold value for the split [float].
        - Input data to be split [ndarray].

    Returns:
        Left and right child nodes after the split.

    Details:
        - Convert the data to a list of instances
        - Iterate over each instance in the data
            - If the value of the feature at the given index is less than the threshold, append the instance to the left child
            - Otherwise, append the instance to the right child
        - Convert the left and right child nodes to numpy arrays
    
    Returns:
        left and right child nodes
        """
        instances = data.tolist()
        left_child = []
        right_child = []
        for row in instances:
            if row[feature_idx] < threshold:
                left_child.append(row)
            else:
                right_child.append(row)
        left_child = np.array(left_child)
        right_child = np.array(right_child)

        return left_child, right_child


    def find_best_split(self, data):
        """ Find the best split for the given data based on the minimum Gini index.

    Parameters:
        Input data to find the best split for [ndarray].

    Returns:
        Best split node [dict]

    Details:
        - Get the number of features in the data
        - Initialize the minimum Gini index to 1000 (a high value)
        - Iterate through each feature and each row in the data
            - Get the value of the current feature and split the data into left and right child nodes
            - Calculate the Gini index for the child nodes using the calculate_gini_index method
            - If the calculated Gini index is lower than the current minimum Gini index, update the min Gini index,\\
                best feature index, best feature value, and best child nodes
        - Create a node dictionary with the best feature index, best feature value, and best child nodes
        """
        num_of_features = len(data[0]) - 1
        min_gini_index = 1000
        best_feature_idx = 0
        best_feature_value = 0
        for feature_idx in range(num_of_features):
            for row in data:
                value = row[feature_idx]
                left_child, right_child = self.apply_split(feature_idx, value, data)
                child_nodes = [left_child, right_child]
                gini_index = self.calculate_gini_index(child_nodes)
                if gini_index < min_gini_index:
                    min_gini_index = gini_index
                    best_feature_idx = feature_idx
                    best_feature_value = value
                    best_child_nodes = child_nodes
        node = {"feature": best_feature_idx, "value": best_feature_value, "children": best_child_nodes}

        return node

    def calculate_class_value(self, node):
        """ Compute the class value for a given node.

    Parameters:
        Input node to calculate the class value for [ndarray].

    Returns:
        The the most common class value from the occurrence count.

    Details:
        - Iterate over each row in the node and append to the class_values list 
            the class value (the last element in each row of the node array).
        - Count the occurrence of each class value using the Counter class.
        """
        class_values = []
        for row in node:
            class_values.append(row[-1])
        occurrence_count = Counter(class_values)
        
        return occurrence_count.most_common(1)[0][0]

In [5]:
@update_class()
class DecisionTree:
    def recursive_split(self, node, depth):
        """ Split the decision tree nodes based on the given node and depth.

    Parameters:
        - Current node to split [dict]
        - Current depth of the node in the decision tree [int]

    Details:
        - Extract the left and right child nodes from the current node
        - Remove the "children" key from the current node
        - Check if either the left or right child node is empty (has size 0)
            - If the left child is empty, calculate the class value for the right child and terminate both child nodes
            - If the right child is empty, calculate the class value for the left child and terminate both child nodes
        - Check if the tree has reached the maximum depth
            - If the maximum depth is reached, calculate the class values for both child nodes and terminate them
        - If the tree has not reached the maximum depth, continue splitting the left and right child nodes
            - If the size of the left child is less than or equal to the minimum node size, calculate the class value for\\
                the left child and terminate it
            - Otherwise, find the best split for the left child, recursively split its child nodes, and assign the result\\
                to the "left" key of the current node
            - If the size of the right child is less than or equal to the minimum node size, calculate the class value for\\
                the right child and terminate it
            - Otherwise, find the best split for the right child, recursively split its child nodes, and assign the result\\
                to the "right" key of the current node
        """
        left_child, right_child = node["children"]
        del node["children"]
        if left_child.size == 0:
            class_value = self.calculate_class_value(right_child)
            node["left"] = node["right"] = {"class_value": class_value, "depth": depth}
            return
        elif right_child.size == 0:
            class_value = self.calculate_class_value(left_child)
            node["left"] = node["right"] = {"class_value": class_value, "depth": depth}
            return
        if depth >= self.max_depth:
            class_value = self.calculate_class_value(left_child)
            node["left"] = {"class_value": class_value, "depth": depth}
            class_value = self.calculate_class_value(right_child)
            node["right"] = {"class_value": class_value, "depth": depth}
            return
        if len(left_child) <= self.min_node_size:
            class_value = self.calculate_class_value(left_child)
            node["left"] = {"class_value": class_value, "depth": depth}
        else:
            node["left"] = self.find_best_split(left_child)
            self.recursive_split(node["left"], depth + 1)
        if len(right_child) <= self.min_node_size:
            class_value = self.calculate_class_value(right_child)
            node["right"] = {"class_value": class_value, "depth": depth}
        else:
            node["right"] = self.find_best_split(right_child)
            self.recursive_split(node["right"], depth + 1)

In [6]:
@update_class()
class DecisionTree:
    def train(self, X):
        """ Train the decision tree using the provided input data.
    
    Parameters:
        X : Input data for decision tree training [ndarray]

    Details:
        - Create the initial node by finding the best split for the input data
        - Recursively split the initial node and generate the rest of the decision tree
        - Set the final_tree attribute to the trained decision tree
        - Return the trained decision tree
    
    Returns:
        Trained decision tree [dict]
    """
        tree = self.find_best_split(X)
        self.recursive_split(tree, 1)
        self.final_tree = tree
        return tree


    def print_decision_tree(self, tree, depth=0):
        """ Print the decision tree structure.

        Args:
            - Decision tree to print [dict]
            - Current depth of the tree node [int]

        Details:
            - Check if the current tree node has a "feature" key.
                - If it does, print the split node information (feature, value, and depth).
                - Recursively call the print_decision_tree method for the left and right child nodes, incrementing the depth.
            - If the current tree node does not have a "feature" key, it is a terminal node.
                - Print the terminal node information (class value and depth).
        """
        if "feature" in tree:
            print(
                "\nSPLIT NODE: feature #{} < {} depth: {}\n".format(
                    tree["feature"], tree["value"], depth
                )
            )
            self.print_decision_tree(tree["left"], depth + 1)
            self.print_decision_tree(tree["right"], depth +1)

In [7]:
@update_class()
class DecisionTree:
    def recursive_split(self, node, depth):
        """ Recursively split the decision tree nodes based on the given node and depth.

        Parameters:
            - Current node to split [dict]
            - Current depth of the node in the decision tree [int]
        
        Details:
            - Extract the left and right child nodes from the current node
            - Remove the "children" key from the current node
            - Check if either the left or right child node is empty (has size 0)
                - If the left child is empty, calculate the class value for the right child and terminate both child nodes
                - If the right child is empty, calculate the class value for the left child and terminate both child nodes
            - Check if the tree has reached the maximum depth
                - If the maximum depth is reached, calculate the class values for both child nodes and terminate them
            - If the tree has not reached the maximum depth, continue splitting the left and right child nodes
                - If the size of the left child is less than or equal to the minimum node size, calculate the class value for\\
                the left child and terminate it
                - Otherwise, find the best split for the left child, recursively split its child nodes, and assign the result\\
                to the "left" key of the current node
                - If the size of the right child is less than or equal to the minimum node size, calculate the class value for\\
                the right child and terminate it
                - Otherwise, find the best split for the right child, recursively split its child nodes, and assign the result\\
                to the "right" key of the current node
        """
        left_child, right_child = node["children"]
        del node["children"]
        if left_child.size == 0:
            class_value = self.calculate_class_value(right_child)
            node["left"] = node["right"] = {"class_value": class_value, "depth": depth}
            return
        elif right_child.size == 0:
            class_value = self.calculate_class_value(left_child)
            node["left"] = node["right"] = {"class_value": class_value, "depth": depth}
            return
        if depth >= self.max_depth:
            class_value = self.calculate_class_value(left_child)
            node["left"] = {"class_value": class_value, "depth": depth}
            class_value = self.calculate_class_value(right_child)
            node["right"] = {"class_value": class_value, "depth": depth}
            return
        if len(left_child) <= self.min_node_size:
            class_value = self.calculate_class_value(left_child)
            node["left"] = {"class_value": class_value, "depth": depth}
        else:
            node["left"] = self.find_best_split(left_child)
            self.recursive_split(node["left"], depth + 1)
        if len(right_child) <= self.min_node_size:
            class_value = self.calculate_class_value(right_child)
            node["right"] = {"class_value": class_value, "depth": depth}
        else:
            node["right"] = self.find_best_split(right_child)
            self.recursive_split(node["right"], depth + 1)

In [8]:
@update_class()
class DecisionTree:
    def predict_single_instance(self, tree, instance):
        """ Predicts the class value for a single instance using the trained decision tree.

        Parameters:
            - Decision tree to use for prediction [dict]
            - Input instance for which to predict the class value [list / ndarray]

        Details:
            Traverse the decision tree to predict the class value for a single instance
            - If the tree is empty (not trained), it raises a ValueError
            - If the current tree node is a split node (the "feature" key)
                it compares the value of the instance for the corresponding feature with the split value
            - If the value is less than the split value, it recursively calls 'predict_single_instance' on the left child node
            - If the value is greater than or equal to the split value, it recursively calls 'predict_single_instance' on the right child node
            - If the current tree node is a terminal node (leaf node), it returns the class value stored in the "class_value" key
        
        Returns:
            The predicted class value for the given instance [int]
        """
        if not tree:
            print("ERROR: Please train the decision tree first")
            return -1
        if "feature" in tree:
            if instance[tree["feature"]] < tree["value"]:
                return self.predict_single_instance(tree["left"], instance)
            else:
                return self.predict_single_instance(tree["right"], instance)
        else:
            return tree["class_value"]


    def predict(self, X):
        """ Predict the class values for multiple instances using the trained decision tree.

        Parameters:
            X : Input instances [list / ndarray]

        Details:
            - Iterate over the instances in X and calls 'predict_single_instance' to predict the class value for each instance
                - The predicted class value is appended to the list of predictions
                - Convert the list of predictions to a numpy array and returns it as the predicted class values

        Returns:
            Predicted class values for the given instances [ndarray]
        """
        y_predictions = []
        for row in X:
            y_predictions.append(self.predict_single_instance(self.final_tree, row))
        return np.array(y_predictions)

In [9]:
train_data = np.loadtxt("./data_trees/data.txt", delimiter=",")
train_y = np.loadtxt("./data_trees/targets.txt")

## Build tree
dt = DecisionTree(5, 1)
tree = dt.train(train_data)
y_pred = dt.predict(train_data)
print(f"Accuracy: {sum(y_pred == train_y) / train_y.shape[0]}")

Accuracy: 0.9888888888888889


In [11]:
dt.print_decision_tree(tree)


SPLIT NODE: feature #1 < -0.1189 depth: 0


SPLIT NODE: feature #0 < 0.3505 depth: 1


SPLIT NODE: feature #1 < -1.8785 depth: 2


SPLIT NODE: feature #0 < -0.2029 depth: 3


SPLIT NODE: feature #0 < 0.0555 depth: 3


SPLIT NODE: feature #0 < -0.3927 depth: 4


SPLIT NODE: feature #0 < 1.3432 depth: 2


SPLIT NODE: feature #1 < -2.1079 depth: 3


SPLIT NODE: feature #0 < 0.8358 depth: 4


SPLIT NODE: feature #0 < 1.1107 depth: 4


SPLIT NODE: feature #0 < 1.4758 depth: 3


SPLIT NODE: feature #0 < 1.3432 depth: 4


SPLIT NODE: feature #0 < 1.4758 depth: 4


SPLIT NODE: feature #1 < 1.8649 depth: 1


SPLIT NODE: feature #0 < 1.7032 depth: 2


SPLIT NODE: feature #0 < 0.1166 depth: 3


SPLIT NODE: feature #0 < -0.5498 depth: 4


SPLIT NODE: feature #0 < 0.3832 depth: 4


SPLIT NODE: feature #0 < 2.3009 depth: 3


SPLIT NODE: feature #0 < 2.3009 depth: 4


SPLIT NODE: feature #0 < 0.0988 depth: 2


SPLIT NODE: feature #1 < 3.1421 depth: 3


SPLIT NODE: feature #0 < -0.0382 depth: 4


SPL